# Boosting SLP

## Import Problem

In [1]:
import pycutest as pc
import numpy as np
import os
from collections import OrderedDict
import autograd.numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from autograd import grad
from collections import Counter, defaultdict
from itertools import compress
from scipy.io import loadmat
from scipy.optimize import linear_sum_assignment
from scipy.sparse.linalg import svds
from scipy.special import huber
from sklearn.preprocessing import StandardScaler
from time import process_time
from boosting_utils import *
import warnings
warnings.simplefilter('ignore')
import logging
import pickle
import time
# from cuter_util import Cuter
# from linear_solver import linearSolveTrustRegion
from param import DustParam

/home/mcq/Documents/Github/venv/lib/python3.10/site-packages/pycutest/__init__.py:28: RuntimeWarning: the PYCUTEST_CACHE environment variable is not set; current folder will be used for caching.
  warnings.warn("the PYCUTEST_CACHE environment variable is not set; current folder will be used for caching.", RuntimeWarning)


In [2]:
pc.print_available_sif_params('HS20')
p = pc.import_problem('HS20')
p_properties = pc.problem_properties('HS20')
print(p)
print(p_properties)
p.report()

Parameters available for problem HS20:
End of parameters for problem HS20
CUTEst problem HS20 (default params) with 2 variables and 3 constraints
{'objective': 'sum of squares', 'constraints': 'quadratic', 'regular': True, 'degree': 2, 'origin': 'academic', 'internal': False, 'n': 2, 'm': 3}


{'f': 0,
 'g': 0,
 'H': 0,
 'Hprod': 0,
 'c': 0,
 'cg': 0,
 'cH': 0,
 'tsetup': 0.0,
 'trun': 0.0}

In [3]:
setup_args_dict = cuter_extra_setup_args(p)
print (setup_args_dict)

{'x': array([-2.,  1.]), 'bl': array([-5.e-01, -1.e+20]), 'bu': array([5.e-01, 1.e+20]), 'v': array([0., 0., 0.]), 'cl': array([0., 0., 0.]), 'cu': array([1.e+20, 1.e+20, 1.e+20]), 'equatn': array([False, False, False]), 'linear': array([False, False, False]), 'bl_flag': array([ True, False]), 'bu_flag': array([ True, False]), 'adjusted_equatn': array([False, False, False, False, False]), 'iequatn': array([ True,  True,  True]), 'inequality_lower': array([ True,  True,  True]), 'inequality_upper': array([False, False, False]), 'is_lower_bound_only_constr': array([ True,  True,  True]), 'is_double_bound_constr': array([False, False, False])}


In [4]:
linearSolveTrustRegion(p, DustParam, setup_args_dict)

IndexError: boolean index did not match indexed array along dimension 0; dimension is 3 but corresponding boolean dimension is 5